In [1]:
%load_ext Cython
import sys
sys.path[0] = ('/home/bram/ANTS/')

In [9]:
%%cython -a
from cythonic.core.map cimport MeshMap, GaussMap
from libc.math cimport sqrt as csqrt, M_PI
cimport numpy as np
import numpy as np

map = MeshMap(np.array([100,100],dtype = np.float_),0.25)
cdef double sigmax=1.0, sigmay=1.0, mux=0.0, muy=0.0, sigmaxy=0.0
cdef double[:,::1] biv_normal(double [:,::1] X, double[:,::1] Y):
    cdef double rho = sigmaxy/(sigmax*sigmay)
    cdef double[:,::] nom = -(np.array(X)-mux)**2/sigmax**2 - (np.array(Y)-muy)**2/sigmay**2 + 2*rho*(np.array(X)-mux)*(np.array(Y)-muy)/(sigmax*sigmay)
    cdef double denom = 2*M_PI*sigmax*sigmay*csqrt(1-rho**2)
    return np.exp(np.array(nom)/(2*(1-rho**2))) / denom

def pybiv_normal(X,Y):
    Xmu = X-mux
    Ymu = Y-muy

    rho = sigmaxy/(sigmax*sigmay)
    z = Xmu**2/sigmax**2 + Ymu**2/sigmay**2 - 2*rho*Xmu*Ymu/(sigmax*sigmay)
    denom = 2*np.pi*sigmax*sigmay*np.sqrt(1-rho**2)
    return np.exp(-z/(2*(1-rho**2))) / denom

from time import time
cdef double tic, toc
cdef double[:,::1] x
cdef int i, N = 100
tic = time()

for i in range(N):
    x = np.array(biv_normal(map.mesh_x,map.mesh_y))
toc = time()
print("c-avg: {} msec".format((toc-tic)*1000))
tic = time()

for i in range(N):
    y = np.array(pybiv_normal(np.array(map.mesh_x),np.array(map.mesh_y)))
toc = time()
print("py-avg: {} msec".format((toc-tic)*1000))

tic=time()
G = GaussMap(R=50., resolution= 0.25, covariance = 1.)
for i in range(N):
    z = G.bivariate_normal()
toc = time()
print("mesh-avg: {} msec".format((toc-tic)*1000))

# Xmu = X-mux
# Ymu = Y-muy

# rho = sigmaxy/(sigmax*sigmay)
# z = Xmu**2/sigmax**2 + Ymu**2/sigmay**2 - 2*rho*Xmu*Ymu/(sigmax*sigmay)
# denom = 2*np.pi*sigmax*sigmay*np.sqrt(1-rho**2)
# return np.exp(-z/(2*(1-rho**2))) / denom



c-avg: 1151.7534255981445 msec
py-avg: 1420.9392070770264 msec


TypeError: unsupported operand type(s) for -: 'cythonic.core.map._memoryviewslice' and 'float'